In [7]:
import torch
import torch.nn.functional as F
from layers.MOE_dispatcher import MOEDispatcher
import pickle
data = pickle.load(open('./dataset/Stanford/Stanford_Nova_Regular_191.pkl', 'rb'))
# data = pickle.load(open('./dataset/total_MICH/MICH_18H_pouch_NMC_45C_50-100_0.2-1.5C.pkl', 'rb'))
print(data.keys())
print(data['cathode_material'])
print(data['anode_material'])
print(data['form_factor'])
top_k = 2
eps = 1e-9
logits = torch.rand(3,4)
logits

dict_keys(['cell_id', 'cycle_data', 'form_factor', 'anode_material', 'cathode_material', 'electrolyte_material', 'nominal_capacity_in_Ah', 'depth_of_charge', 'depth_of_discharge', 'already_spent_cycles', 'max_voltage_limit_in_V', 'min_voltage_limit_in_V', 'max_current_limit_in_A', 'min_current_limit_in_A', 'reference', 'description', 'charge_protocol', 'discharge_protocol', 'SOC_interval'])
Li[Ni0.5Mn0.3Co0.2]O2
graphite
cylindrical


tensor([[0.4424, 0.5810, 0.0740, 0.1841],
        [0.9098, 0.7994, 0.9321, 0.4260],
        [0.6099, 0.8680, 0.7366, 0.6580]])

In [2]:
_, indices = torch.topk(logits, top_k, dim=1)
# Create a mask where only the top-K values will be kept
mask = torch.zeros_like(logits, dtype=torch.bool)
# Scatter the mask at the indices of the top-K values
mask.scatter_(1, indices, 1) # 0 indicates mask
logits = F.softmax(logits, dim=1) # [B, num_experts]
raw_logits = logits.clone()
# logits.masked_fill_(mask==0, 0) # [B, num_experts]
logits = logits * mask
de_norm = torch.sum(logits, dim=1) + eps
logits = logits / de_norm.unsqueeze(-1)
logits

tensor([[0.4841, 0.5159, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4994, 0.5006],
        [0.5037, 0.0000, 0.0000, 0.4963]])

In [3]:
my_dispatcher = MOEDispatcher(2, logits, 2)

In [4]:
print("_batch_index", my_dispatcher._batch_index)
print("_nonzero_gates", my_dispatcher._nonzero_gates)
print("_part_sizes", my_dispatcher._part_sizes)
print(my_dispatcher.dispatch())

_batch_index tensor([0, 2, 0, 1, 1, 2])
_nonzero_gates tensor([[0.4841],
        [0.5037],
        [0.5159],
        [0.4994],
        [0.5006],
        [0.4963]])
_part_sizes [2, 1, 1, 2]
(tensor([0, 2]), tensor([0]), tensor([1]), tensor([1, 2]))


In [5]:
def rearrange_tensor(X, _batch_index):
    # Get the batch size B
    B = len(_batch_index) // 2
    
    # Create an index tensor to rearrange X
    # This will have shape [B, 2] where each row contains indices of the two representations
    indices = torch.zeros((B, 2), dtype=torch.long)
    
    # Fill in the indices array
    for i in range(B):
        indices[i] = torch.where(_batch_index == i)[0]
    
    # Use advanced indexing to gather the pairs
    Y = X[indices]
    
    return Y

_batch_index = torch.FloatTensor([1,0,2,2,0,1])
X = torch.rand(6,12)
rearrange_tensor(X,_batch_index).shape

torch.Size([3, 2, 12])